In [1]:
import gc
import torch

import transformers
from datasets import load_dataset
from peft import LoraConfig
from trl import DPOTrainer, DPOConfig, SFTConfig, SFTTrainer
import bitsandbytes as bnb

from tqdm import tqdm
import numpy as np
import warnings

warnings.filterwarnings("ignore")

In [2]:
import os
os.environ["VLLM_USE_V1"] = '0'

In [3]:
from unsloth import FastLanguageModel, is_bfloat16_supported
import torch
max_seq_length = 1024
lora_rank = 16

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "Qwen/Qwen2.5-3B-Instruct",
    max_seq_length = max_seq_length,
    load_in_4bit = True, # False for LoRA 16bit
    fast_inference = True, # Enable vLLM fast inference
    max_lora_rank = lora_rank,
    gpu_memory_utilization = 0.5, # Reduce if out of memory
)

model = FastLanguageModel.get_peft_model(
    model,
    r = lora_rank,
    target_modules = [
        "q_proj", "k_proj", "v_proj", "o_proj",
        "gate_proj", "up_proj", "down_proj",
    ],
    lora_alpha = lora_rank,
    use_gradient_checkpointing = "unsloth",
    random_state = 2811,
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
Unsloth: Failed to patch SmolVLMForConditionalGeneration forward function.
🦥 Unsloth Zoo will now patch everything to make training faster!
INFO 05-06 23:11:38 [__init__.py:239] Automatically detected platform cuda.
==((====))==  Unsloth 2025.4.1: Fast Qwen2 patching. Transformers: 4.51.3. vLLM: 0.8.4.
   \\   /|    NVIDIA GeForce RTX 4070 Ti SUPER. Num GPUs = 1. Max memory: 15.992 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 8.9. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.29.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Unsloth: vLLM loading unsloth/qwen2.5-3b-instruct-unsloth-bnb-4bit with actual GPU utilization = 45.97%
Unsloth: Your GPU has CUDA compute capability 8.9 with VRAM = 15.99 GB.
Unsloth: Using conservativeness = 1.0. Chun

Loading safetensors checkpoint shards:   0% Completed | 0/1 [00:00<?, ?it/s]


Loading safetensors checkpoint shards:   0% Completed | 0/1 [00:00<?, ?it/s]


INFO 05-06 23:12:01 [punica_selector.py:18] Using PunicaWrapperGPU.
INFO 05-06 23:12:01 [model_runner.py:1146] Model loading took 2.2829 GiB and 5.380828 seconds
INFO 05-06 23:12:02 [worker.py:267] Memory profiling takes 1.02 seconds
INFO 05-06 23:12:02 [worker.py:267] the current vLLM instance can use total_gpu_memory (15.99GiB) x gpu_memory_utilization (0.46) = 7.35GiB
INFO 05-06 23:12:02 [worker.py:267] model weights take 2.28GiB; non_torch_memory takes 0.05GiB; PyTorch activation peak memory takes 1.05GiB; the rest of the memory reserved for KV Cache is 3.98GiB.
INFO 05-06 23:12:02 [executor_base.py:112] # cuda blocks: 7238, # CPU blocks: 3640
INFO 05-06 23:12:02 [executor_base.py:117] Maximum concurrency for 1024 tokens per request: 113.09x
INFO 05-06 23:12:03 [model_runner.py:1456] Capturing cudagraphs for decoding. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI. If 

Capturing CUDA graph shapes:   0%|          | 0/27 [00:00<?, ?it/s]

INFO 05-06 23:12:21 [model_runner.py:1598] Graph capturing finished in 16 secs, took 0.57 GiB
INFO 05-06 23:12:21 [llm_engine.py:449] init engine (profile, create kv cache, warmup model) took 20.35 seconds


Unsloth 2025.4.1 patched 36 layers with 36 QKV layers, 36 O layers and 36 MLP layers.


In [4]:
dataset = load_dataset("thainq107/Vi-Alpaca-Preference")
dataset, dataset["train"][0]

(DatasetDict({
     train: Dataset({
         features: ['id', 'question', 'chosen', 'rejected'],
         num_rows: 65017
     })
     test: Dataset({
         features: ['id', 'question', 'chosen', 'rejected'],
         num_rows: 2000
     })
 }),
 {'id': 'alpaca-7294',
  'question': 'Xác định và sửa lỗi ngữ pháp.\n\nTôi đã đi đến cửa hàng.',
  'chosen': 'Không có lỗi ngữ pháp. Câu này đã chính xác.',
  'rejected': 'Câu này không có lỗi ngữ pháp.'})

In [5]:
# QLoRA configuration
peft_config = LoraConfig(
    r=16,
    lora_alpha=16,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=[
        "q_proj",
        "k_proj",
        "v_proj",
        "o_proj",
        "gate_proj",
        "up_proj",
        "down_proj"
    ]
)

## **SFT**

In [6]:
def formatting_prompt_with_chat_template(example):
    conversation = [
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": example["question"]},
        {"role": "assistant", "content": example["chosen"]},
    ]
    prompt = tokenizer.apply_chat_template(
        conversation, tokenize=False, add_generation_prompt=False
    )
    return prompt

In [7]:
hyperparameters = {
    "per_device_train_batch_size": 4,
    "gradient_accumulation_steps": 8,
    "gradient_checkpointing": True,
    "learning_rate": 3e-5,
    "logging_steps": 200,
    "num_train_epochs": 1,
    "save_strategy": "no",
    "overwrite_output_dir": True,
    "optim": "paged_adamw_8bit",
    "warmup_steps": 200,
    "bf16": True,
}
MAX_LENGTH = 1024

In [8]:
SFT_OUTPUT_DIR = f"Qwen2.5-3B-Instruct-sft"

sft_config = SFTConfig(
    **{ **hyperparameters, "output_dir":
       SFT_OUTPUT_DIR , "max_seq_length": MAX_LENGTH}
)

os.environ['UNSLOTH_RETURN_LOGITS'] = '1'

sft_trainer = SFTTrainer(
    model=model,
    peft_config=peft_config,
    processing_class=tokenizer,
    args=sft_config,
    train_dataset=dataset['train'],
    formatting_func=formatting_prompt_with_chat_template,
)

sft_trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 65,017 | Num Epochs = 1 | Total steps = 2,031
O^O/ \_/ \    Batch size per device = 4 | Gradient accumulation steps = 8
\        /    Data Parallel GPUs = 1 | Total batch size (4 x 8 x 1) = 32
 "-____-"     Trainable parameters = 29,933,568/3,000,000,000 (1.00% trained)
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Currently logged in as: tridungluong123 (tridungluong123-university) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
200,1.549000


KeyboardInterrupt: 

In [ ]:
sft_trainer.push_to_hub("Qwen2.5-3B-instruct-sft-vi-alpaca", token="###")

## **DPO**

In [ ]:
def convert_to_conversational_preference_format(example):
    return {
        "id": example["id"],
        "prompt": [{"role": "system",
                    "content": "You are a helpful assistant."},
                   {"role": "user",
                    "content": example["question"]}],
        "chosen": [{"role": "assistant",
                    "content": example["chosen"]}],
        "rejected": [{"role": "assistant",
                      "content": example["rejected"]}],
    }

dpo_dataset = dataset.map(convert_to_conversational_preference_format)


In [ ]:
dpo_dataset['train'][0]

{'id': 'alpaca-7294',
 'question': 'Xác định và sửa lỗi ngữ pháp.\n\nTôi đã đi đến cửa hàng.',
 'chosen': [{'content': 'Không có lỗi ngữ pháp. Câu này đã chính xác.',
   'role': 'assistant'}],
 'rejected': [{'content': 'Câu này không có lỗi ngữ pháp.',
   'role': 'assistant'}],
 'prompt': [{'content': 'You are a helpful assistant.', 'role': 'system'},
  {'content': 'Xác định và sửa lỗi ngữ pháp.\n\nTôi đã đi đến cửa hàng.',
   'role': 'user'}]}

In [ ]:
hyperparameters = {
    "per_device_train_batch_size": 8,
    "gradient_accumulation_steps": 8,
    "gradient_checkpointing": True,
    "learning_rate": 3e-5,
    "logging_steps": 200,
    "num_train_epochs": 2,
    "save_strategy": "no",
    "overwrite_output_dir": True,
    "optim": "paged_adamw_8bit",
    "warmup_steps": 200,
    "bf16": True,
}
MAX_LENGTH = 512

In [ ]:
# Use wandb
import wandb
wandb.init(
    project="vi-alpaca-preference",
    name="Qwen2.5-3B-4bit-dpo"
)


In [ ]:
DPO_OUTPUT_DIR = f"Qwen2.5-3B-Instruct-dpo"
dpo_args = DPOConfig(
    **{ **hyperparameters, "output_dir":
       DPO_OUTPUT_DIR, "max_length": MAX_LENGTH }
)

dpo_trainer = DPOTrainer(
    model,
    args=dpo_args,
    train_dataset=dpo_dataset['train'],
    processing_class=tokenizer,
    peft_config=peft_config,
)
dpo_trainer.train()


wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


Step,Training Loss
200,0.496700
400,0.303000
600,0.295100
800,0.283300
1000,0.272800
1200,0.270000
1400,0.250200
1600,0.253900
1800,0.258600
2000,0.246600


TrainOutput(global_step=2032, training_loss=0.2921085744861543, metrics={'train_runtime': 37665.8776, 'train_samples_per_second': 3.452, 'train_steps_per_second': 0.054, 'total_flos': 0.0, 'train_loss': 0.2921085744861543, 'epoch': 2.0})

In [ ]:
dpo_trainer.push_to_hub("Qwen2.5-3B-instruct-dpo-vi-alpaca", token="###")